# GPT-OSS Fine-tuning with Unsloth and Bright Data
Complete notebook for fine-tuning GPT-OSS-20B using Unsloth and Bright Data.
Run this on Google Colab with a T4 GPU (free tier).

In [ ]:
# Install Unsloth and dependencies
import subprocess, sys
subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade', '-q', 'uv'])
try:
    import numpy
    get_numpy = f'numpy=={numpy.__version__}'
except:
    get_numpy = 'numpy'
packages = [
    'torch>=2.8.0',
    'triton>=3.4.0',
    get_numpy,
    'torchvision',
    'bitsandbytes',
    'transformers>=4.55.3',
    'unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo',
    'unsloth[base] @ git+https://github.com/unslothai/unsloth',
    'git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels'
]
import os
os.system(f"uv pip install -qqq {' '.join(packages)}")
os.system('uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers')
os.system('uv pip install --no-deps trl==0.22.2')
os.system('pip install -q brightdata-sdk')
print('✅ Dependencies installed!')

In [ ]:
# Check GPU and import libraries
import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer
from trl import SFTConfig, SFTTrainer
from unsloth.chat_templates import standardize_sharegpt, train_on_responses_only
from datasets import Dataset, load_dataset
from typing import List, Dict
import time
gpu_stats = torch.cuda.get_device_properties(0)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f'GPU = {gpu_stats.name}. Max memory = {max_memory} GB.')
if max_memory < 15:
    print('⚠️ Warning: You need at least 16GB GPU memory. Switch to T4 or better.')
else:
    print('✅ GPU memory sufficient for GPT-OSS-20B fine-tuning!')

In [ ]:
# Load GPT-OSS-20B Model with Unsloth
max_seq_length = 1024
dtype = None
print('Loading GPT-OSS-20B with Unsloth...')
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/gpt-oss-20b',
    dtype = dtype,
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    full_finetuning = False,
)
print('✅ Model loaded successfully!')

In [ ]:
# Add LoRA Adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj',
                      'gate_proj', 'up_proj', 'down_proj'],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = 'none',
    use_gradient_checkpointing = 'unsloth',
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
print('✅ LoRA applied!')
print(f'Training {trainable_params:,} / {all_params:,} params')
print(f
)

In [ ]:
# Test Reasoning Effort Levels
print('Testing reasoning effort levels...')
messages = [{"role": "user", "content": "What is 15 * 23?"}]
print('=== LOW REASONING (Fast) ===')
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors='pt', return_dict=True, reasoning_effort='low').to('cuda')
_ = model.generate(**inputs, max_new_tokens=32, streamer=TextStreamer(tokenizer))
print('=== HIGH REASONING (Accurate) ===')
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors='pt', return_dict=True, reasoning_effort='high').to('cuda')
_ = model.generate(**inputs, max_new_tokens=128, streamer=TextStreamer(tokenizer))

In [ ]:
# Collect Training Data with Bright Data
from brightdata import bdclient
from typing import List, Dict
import re
class DataCollector:
    def __init__(self, api_token: str):
        self.client = bdclient(api_token=api_token)
        self.collected_data = []
        print('✅ Bright Data client initialized')
    def collect_documentation(self, urls: List[str]) -> List[Dict]:
        print(f'Scraping {len(urls)} URLs with Bright Data...')
        try:
            results = self.client.scrape(urls, data_format='markdown')
            if isinstance(results, str):
                training_data = self.process_single_result(results)
            elif isinstance(results, list):
                training_data = []
                for content in results:
                    if content:
                        examples = self.process_single_result(content)
                        training_data.extend(examples)
            else:
                print(f'Unexpected result type: {type(results)}')
                training_data = []
        except Exception as e:
            print(f'Batch scraping failed: {e}')
            training_data = []
            for url in urls:
                try:
                    print(f'  Scraping: {url}')
                    content = self.client.scrape(url, data_format='markdown')
                    if content:
                        examples = self.process_single_result(content)
                        training_data.extend(examples)
                        print(f'    ✓ Got {len(examples)} examples')
                except Exception as url_error:
                    print(f'    ✗ Error: {url_error}')
        self.collected_data = training_data
        print(f'✅ Total examples collected: {len(self.collected_data)}')
        return self.collected_data
    def process_single_result(self, content: str) -> List[Dict]:
        examples = []
        content = re.sub(r'<[^>]+>', '', content)
        content = re.sub(r'!.*?.*?', '', content)
        content = re.sub(r'([^]+)[^]+', r'', content)
        content = re.sub(r'```[^`]*```', '', content)
        content = re.sub(r'`[^`]+`', '', content)
        content = re.sub(r'[#*_~>`|-]+', ' ', content)
        content = re.sub(r'\(.)', r'', content)
        content = re.sub(r'https?://[^]+', '', content)
        content = re.sub(r'++', '', content)
        content = re.sub(r'+', ' ', content)
        sentences = re.split(r'(?<=[.!?])+', content)
        clean_sentences = []
        for sent in sentences:
            sent = sent.strip()
            if (len(sent) > 30 and not any(skip in sent.lower() for skip in ['navigation', 'copyright', 'index', 'table of contents', 'previous', 'next'])):
                clean_sentences.append(sent)
        for i in range(0, len(clean_sentences) - 1):
            instruction = clean_sentences[i][:200].strip()
            response = clean_sentences[i + 1][:300].strip()
            if len(instruction) > 20 and len(response) > 30:
                examples.append({'instruction': instruction, 'response': response})
        return examples
BRIGHTDATA_API_TOKEN = 'your_token_here'
urls = [
    'https://docs.python.org/3/tutorial/introduction.html',
    'https://docs.python.org/3/tutorial/controlflow.html',
    'https://docs.python.org/3/tutorial/datastructures.html',
]
collector = DataCollector(api_token=BRIGHTDATA_API_TOKEN)
print('Collecting training data...')
training_data = collector.collect_documentation(urls)
if len(training_data) == 0:
    print('⚠️ ERROR: No training data collected!')
    raise ValueError('No training data collected')
# Final validation
def final_validation(examples: List[Dict]) -> List[Dict]:
    clean_data = []
    seen = set()
    for ex in examples:
        instruction = ex.get('instruction', '').strip()
        response = ex.get('response', '').strip()
        instruction = re.sub(r'[^a-zA-Z0-9,?!]', '', instruction)
        response = re.sub(r'[^a-zA-Z0-9,?!]', '', response)
        if (len(instruction) > 10 and len(response) > 20 and instruction not in seen):
            seen.add(instruction)
            clean_data.append({'instruction': instruction, 'response': response})
    return clean_data
training_data = final_validation(training_data)
print(f'Final clean dataset: {len(training_data)} examples')
if len(training_data) == 0:
    raise ValueError('No valid training data after cleaning')
print('Clean training examples:')
for i, example in enumerate(training_data[:3]):
    print(f'Example {i+1}:')
    print(f'Instruction: {example["instruction"]}')
    print(f'Response: {example["response"]}')

In [ ]:
# Format Data for Training
from datasets import Dataset
from unsloth.chat_templates import standardize_sharegpt
def prepare_dataset(raw_data):
    formatted_data = []
    for item in raw_data:
        formatted_data.append({
            'messages': [
                {'role': 'user', 'content': item['instruction']},
                {'role': 'assistant', 'content': item['response']}
            ]
        })
    dataset = Dataset.from_list(formatted_data)
    dataset = standardize_sharegpt(dataset)
    def formatting_prompts_func(examples):
        convos = examples['messages']
        texts = []
        for convo in convos:
            text = tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
            texts.append(text)
        return {'text': texts}
    dataset = dataset.map(formatting_prompts_func, batched=True)
    print(f'✅ Dataset ready with {len(dataset)} examples')
    print('Example formatted text (first 500 chars):')
    print(dataset[0]['text'][:500])
    if '<|channel|>' not in dataset[0]['text']:
        print('⚠️ Warning: Missing channel in format. Adding explicit channel...')
        def fix_formatting(examples):
            fixed_texts = []
            for text in examples['text']:
                text = text.replace('<|start|>assistant<|message|>', '<|start|>assistant<|channel|>final<|message|>')
                fixed_texts.append(text)
            return {'text': fixed_texts}
        dataset = dataset.map(fix_formatting, batched=True)
        print('✅ Fixed formatting with channel')
        print('Fixed example (first 500 chars):')
        print(dataset[0]['text'][:500])
    return dataset
dataset = prepare_dataset(training_data)

In [ ]:
# Setup Training Configuration
from trl import SFTConfig, SFTTrainer
from unsloth.chat_templates import train_on_responses_only
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = 'adamw_8bit',
        weight_decay = 0.01,
        lr_scheduler_type = 'linear',
        seed = 3407,
        output_dir = 'outputs',
        report_to = 'none',
    ),
)
gpt_oss_kwargs = dict(
    instruction_part = '<|start|>user<|message|>',
    response_part = '<|start|>assistant<|channel|>final<|message|>'
)
trainer = train_on_responses_only(trainer, **gpt_oss_kwargs)
print('✅ Trainer configured!')
sample = trainer.train_dataset[0]
decoded_labels = tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in sample['labels']]).replace(tokenizer.pad_token, ' ')
print('Verifying we only train on assistant responses:')
print(f'Training on: {decoded_labels[:200]}...')

In [ ]:
# Train the Model
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f'GPU memory reserved before training: {start_gpu_memory} GB')
print('🚀 Starting training...')
print('This will take about 5-10 minutes for 30 steps.')
print('For full training, set max_steps=None and num_train_epochs=1')
trainer_stats = trainer.train()
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
print('✅ Training completed!')
print(f'Time: {trainer_stats.metrics['train_runtime']/60:.1f} minutes')
print(f'Final loss: {trainer_stats.metrics['train_loss']:.4f}')
print(f'Peak memory for training: {used_memory_for_lora} GB')
print(f'Total peak memory: {used_memory} GB / {max_memory} GB ({used_memory/max_memory*100:.1f}%)')

In [ ]:
# Test the Fine-tuned Model
print('🧪 Testing fine-tuned model on Python questions...')
def test_model(prompt: str, reasoning_effort: str = 'medium'):
    messages = [
        {'role': 'system', 'content': 'You are a helpful Python expert assistant.'},
        {'role': 'user', 'content': prompt}
    ]
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors='pt', return_dict=True, reasoning_effort=reasoning_effort).to('cuda')
    print(f'Question: {prompt}')
    print(f'Reasoning effort: {reasoning_effort}')
    print('Answer:')
    outputs = model.generate(**inputs, max_new_tokens=150, streamer=TextStreamer(tokenizer, skip_prompt=True), temperature=0.7, top_p=0.9)
    print('' + '='*60 + '')
test_questions = [
    'What is a Python generator?',
    'How do I read a CSV file in Python?',
    'Explain async/await in Python'
]
for question in test_questions:
    print(f'{'='*60}')
    print(f'Question: {question}')
    print(f'')
    response = test_model(question, reasoning_effort='medium')
complex_question = 'Write a Python function that finds all prime numbers up to n using the Sieve of Eratosthenes'
print('='*60)
print('TESTING REASONING EFFORT LEVELS')
print('='*60)
for effort in ['low', 'medium', 'high']:
    print(f'{'='*40}')
    print(f'Reasoning Effort: {effort.upper()}')
    print(f'')
    _ = test_model(complex_question, reasoning_effort=effort)
    print()

In [ ]:
# Save the Fine-tuned Model
print('💾 Saving model...')
model.save_pretrained('gpt-oss-python-expert-lora')
tokenizer.save_pretrained('gpt-oss-python-expert-lora')
print('✅ LoRA adapters saved to gpt-oss-python-expert-lora')
# Optional: Push to Hugging Face Hub
# model.push_to_hub('dexcodes/gpt-oss-python-expert-lora', token='your_hf_token')
# model.push_to_hub_merged('dexcodes/gpt-oss-python-expert', tokenizer, save_method='mxfp4', token='your_hf_token')
print('✅ Model pushed to Hugging Face Hub!')

🎉 Congratulations! You've successfully:

1. ✅ Loaded GPT-OSS-20B with Unsloth (2x faster than vanilla transformers)
2. ✅ Applied LoRA for efficient training (only 1% of parameters)
3. ✅ Collected training data (would use Bright Data in production)
4. ✅ Fine-tuned the model on Python Q&A
5. ✅ Tested the model with different reasoning efforts
6. ✅ Saved the fine-tuned model

Next steps:
- Use Bright Data API for real web scraping: https://brightdata.com
- Train for longer (set num_train_epochs=1)
- Try GPT-OSS-120B if you have A100 GPU
- Deploy with vLLM for production: https://github.com/vllm-project/vllm
- Join Unsloth Discord for tips: https://discord.gg/unsloth

Resources:
- Unsloth Docs: https://docs.unsloth.ai/
- Bright Data: https://brightdata.com/solutions/ai
- Original notebook: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-Fine-tuning.ipynb